In [1]:
from IPython.display import HTML, display
import pandas as pd

In [2]:
yik_df = pd.read_csv('data/yik_df_1.csv').drop(columns=['Unnamed: 0'])
pd.options.display.max_colwidth = 200

##### Создание словаря по локации ((x,y) : [УИКи])

In [3]:
loc_yik = {}

for i in range(yik_df.shape[0]):
    if yik_df['x'][i] != 0:
        point = (round(yik_df['x'][i], 6), round(yik_df['y'][i], 6))
        if point in loc_yik:
            loc_yik[point].append(yik_df['УИК'][i])
        else:
            loc_yik[point] = [yik_df['УИК'][i]]

In [4]:
yik_points = [{
    'yik': yiks,
    'yik_str': str(yiks).strip('[]'),
    'x': loc[0],
    'y': loc[1]}
    for (loc, yiks) in loc_yik.items()]

In [5]:
def get_yik_info(n_yik, col_name):
    return round(yik_df[yik_df['УИК'] == n_yik][col_name].iloc[0], 2)

In [6]:
for yik_dict in yik_points:
    yik_dict['yavka'] = [get_yik_info(n_yik, 'Явка%') for n_yik in yik_dict['yik']]

In [7]:
for yik_dict in yik_points:
    yik_dict['yik_str'] = ''
    for i in range(len(yik_dict['yik'])):
        yik_dict['yik_str'] += '<b>УИК {0}</b><br>Явка - {1} %<br>'.format(yik_dict['yik'][i], yik_dict['yavka'][i])
        
    yik_dict['yik_numbers'] = str(yik_dict['yik']).strip('[]')
    
    yik_dict['kol_yiks'] = len(yik_dict['yik'])

In [8]:
add_place_mark_template = """
.add(new ymaps.Placemark([{x}, {y}], {{
    iconContent: '{kol_yiks}',
    balloonContent: '{yik_str}'
}}))
"""

In [9]:
all_places_list = [add_place_mark_template.format(**yik_points[i]) for i in range(len(yik_points))]

In [10]:
import keys

html_str = """
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <title>Быстрый старт. Размещение интерактивной карты на странице</title>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <script src="https://api-maps.yandex.ru/2.1/?apikey={0}&lang=ru_RU" type="text/javascript">
    </script>
    <script type="text/javascript">
        ymaps.ready(init);
        function init(){{
            var myMap = new ymaps.Map("map", {{
                center: [56.04586, 40.535938],
                zoom: 7
            }});
                       
            myMap.geoObjects
                {1}
            ;
        }}
    </script>
</head>

<body>
    <div id="map" style="width: 1000px; height: 600px"></div>
</body>

</html>
"""


# display(HTML(html_str.format(all_places_list[0])))
display(HTML(html_str.format(keys.API_KEY_YANDEX_MAPS ,'\n'.join(all_places_list))))